In [135]:
# Initial imports
import pandas as pd
import plotly.express as px
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
import hvplot.pandas

## Data Preprocessing:

In [136]:
# import csv files:
crypto_df = pd.read_csv("crypto_data.csv",index_col=0)
crypto_df

,CoinName,Algorithm,IsTrading,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,True,PoW/PoS,4.199995e+01,42
365,365Coin,X11,True,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,True,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,True,PoW,NaN,611000
808,808,SHA-256,True,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...,...
XBC,BitcoinPlus,Scrypt,True,PoS,1.283270e+05,1000000
DVTC,DivotyCoin,Scrypt,False,PoW/PoS,2.149121e+07,100000000
GIOT,Giotto Coin,Scrypt,False,PoW/PoS,NaN,233100000
OPSC,OpenSourceCoin,SHA-256,False,PoW/PoS,NaN,21000000


In [137]:
crypto_df.dtypes

CoinName            object
Algorithm           object
IsTrading             bool
ProofType           object
TotalCoinsMined    float64
TotalCoinSupply     object
dtype: object

In [138]:
# check Nan value:
crypto_df.isnull().sum()

CoinName             0
Algorithm            0
IsTrading            0
ProofType            0
TotalCoinsMined    508
TotalCoinSupply      0
dtype: int64

In [139]:
#Remove all cryptocurrencies that aren’t trading:
crypto_df.drop(crypto_df[crypto_df["IsTrading"] ==False].index,inplace=True)

In [140]:
# check value for Algorithm:
crypto_df["Algorithm"].value_counts()

Scrypt                   394
X11                      182
SHA-256                  121
X13                       54
PoS                       42
                        ... 
Cloverhash                 1
T-Inside                   1
Progressive-n              1
VeChainThor Authority      1
Semux BFT consensus        1
Name: Algorithm, Length: 89, dtype: int64

In [141]:
#Remove all cryptocurrencies that don’t have an algorithm defined.
crypto_df.drop(crypto_df[crypto_df["Algorithm"] =="N/A"].index,inplace=True)

In [142]:
#Remove the IsTrading column.
crypto1_df=crypto_df.drop(["IsTrading"],axis=1)
crypto1_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
365,365Coin,X11,PoW/PoS,NaN,2300000000
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
611,SixEleven,SHA-256,PoW,NaN,611000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
...,...,...,...,...,...
SERO,Super Zero,Ethash,PoW,NaN,1000000000
UOS,UOS,SHA-256,DPoI,NaN,1000000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [143]:
#Remove all cryptocurrencies with at least one null value.
crypto2_df=crypto1_df.dropna()
crypto2_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
808,808,SHA-256,PoW/PoS,0.000000e+00,0
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [144]:
# Remove all cryptocurrencies without coins mined.
crypto3_df=crypto2_df.loc[crypto2_df["TotalCoinsMined"] > 0,:]
crypto3_df

,CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,42 Coin,Scrypt,PoW/PoS,4.199995e+01,42
404,404Coin,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,EliteCoin,X13,PoW/PoS,2.927942e+10,314159265359
BTC,Bitcoin,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethereum,Ethash,PoW,1.076842e+08,0
...,...,...,...,...,...
ZEPH,ZEPHYR,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Gapcoin,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,Beldex,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Horizen,Equihash,PoW,7.296538e+06,21000000


In [145]:
#Store the names of all cryptocurrencies:
coin_df=crypto3_df.filter(["CoinName"])
coin_df

,CoinName
42,42 Coin
404,404Coin
1337,EliteCoin
BTC,Bitcoin
ETH,Ethereum
...,...
ZEPH,ZEPHYR
GAP,Gapcoin
BDX,Beldex
ZEN,Horizen


In [146]:
#Remove the CoinName column:
crypto4_df=crypto3_df.drop(["CoinName"],axis=1)
crypto4_df

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply
42,Scrypt,PoW/PoS,4.199995e+01,42
404,Scrypt,PoW/PoS,1.055185e+09,532000000
1337,X13,PoW/PoS,2.927942e+10,314159265359
BTC,SHA-256,PoW,1.792718e+07,21000000
ETH,Ethash,PoW,1.076842e+08,0
...,...,...,...,...
ZEPH,SHA-256,DPoS,2.000000e+09,2000000000
GAP,Scrypt,PoW/PoS,1.493105e+07,250000000
BDX,CryptoNight,PoW,9.802226e+08,1400222610
ZEN,Equihash,PoW,7.296538e+06,21000000


In [147]:
#Create dummies variables for all of the text features, and store the resulting data on a DataFrame named X:
#basesd on csv,we have 2 columns with text.
X=pd.get_dummies(crypto4_df, columns=["Algorithm","ProofType"])
X

,TotalCoinsMined,TotalCoinSupply,Algorithm_1GB AES Pattern Search,Algorithm_536,Algorithm_Argon2d,Algorithm_BLAKE256,Algorithm_Blake,Algorithm_Blake2S,Algorithm_Blake2b,Algorithm_C11,...,ProofType_PoW/PoS,ProofType_PoW/PoS,ProofType_PoW/PoW,ProofType_PoW/nPoS,ProofType_Pos,ProofType_Proof of Authority,ProofType_Proof of Trust,ProofType_TPoS,ProofType_Zero-Knowledge Proof,ProofType_dPoW/PoW
42,4.199995e+01,42,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
404,1.055185e+09,532000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
1337,2.927942e+10,314159265359,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BTC,1.792718e+07,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ETH,1.076842e+08,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
ZEPH,2.000000e+09,2000000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
GAP,1.493105e+07,250000000,0,0,0,0,0,0,0,0,...,1,0,0,0,0,0,0,0,0,0
BDX,9.802226e+08,1400222610,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
ZEN,7.296538e+06,21000000,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0


In [148]:
#standardize all of the data from the X DataFrame:
X_scaled=StandardScaler().fit_transform(X)
X_scaled

array([[-0.11710817, -0.1528703 , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.09396955, -0.145009  , -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [ 0.52494561,  4.48942416, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       ...,
       [-0.09561336, -0.13217937, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11694817, -0.15255998, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ],
       [-0.11710536, -0.15285552, -0.0433963 , ..., -0.0433963 ,
        -0.0433963 , -0.0433963 ]])

## Reducing Data Dimensions Using PCA


In [149]:
# Initialize PCA model
pca = PCA(n_components=3)

In [150]:
# Get 3 principal components for  data.
X_pca = pca.fit_transform(X_scaled)

In [151]:
#dataFrame for principal componets:
df_pca=pd.DataFrame(
            data=X_pca,columns=["PC 1","PC 2","PC 3"],index=crypto4_df.index)
df_pca

,PC 1,PC 2,PC 3
42,-0.337452,1.019204,-0.510428
404,-0.320782,1.019667,-0.510735
1337,2.313123,1.816473,-0.609166
BTC,-0.146745,-1.313571,0.119048
ETH,-0.153041,-1.979435,0.325009
...,...,...,...
ZEPH,2.465506,0.618604,0.230317
GAP,-0.335497,1.019128,-0.510461
BDX,0.325138,-2.292880,0.356592
ZEN,-0.136711,-2.184359,0.377388


## Clustering Cryptocurrencies Using K-means:

In [152]:
# Find the best value for K
inertia = []
k = list(range(1, 11))

# Calculate the inertia for the range of K values
for i in k:
    km = KMeans(n_clusters=i, random_state=0)
    km.fit(df_pca)
    inertia.append(km.inertia_)

# Create the elbow curve
elbow_data = {"k": k, "inertia": inertia}
df_elbow = pd.DataFrame(elbow_data)
df_elbow.hvplot.line(x="k", y="inertia", xticks=k, title="Elbow Curve")

:Curve   [k]   (inertia)

In [153]:
# Initialize the K-means model
model = KMeans(n_clusters=4, random_state=0)

# Fit the model
model.fit(df_pca)

# Predict clusters
predictions = model.predict(df_pca)

# Add the predicted class columns
df_pca["class"] = model.labels_
df_pca

,PC 1,PC 2,PC 3,class
42,-0.337452,1.019204,-0.510428,0
404,-0.320782,1.019667,-0.510735,0
1337,2.313123,1.816473,-0.609166,0
BTC,-0.146745,-1.313571,0.119048,1
ETH,-0.153041,-1.979435,0.325009,1
...,...,...,...,...
ZEPH,2.465506,0.618604,0.230317,0
GAP,-0.335497,1.019128,-0.510461,0
BDX,0.325138,-2.292880,0.356592,1
ZEN,-0.136711,-2.184359,0.377388,1


In [161]:
#Create a new DataFrame by make join  by outer:
clustered1_df=crypto4_df.join(df_pca,how="outer")
clustered1_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.337452,1.019204,-0.510428,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.320782,1.019667,-0.510735,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.313123,1.816473,-0.609166,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.146745,-1.313571,0.119048,1
ETH,Ethash,PoW,1.076842e+08,0,-0.153041,-1.979435,0.325009,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.168538,-1.159298,-0.000968,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.390473,1.315974,-0.455447,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.148064,-2.184012,0.359271,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.151482,-1.979503,0.324983,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.136710,-2.184359,0.377388,1


In [164]:
#create a new dataFrame with coin:
clustered_df=clustered1_df.join(coin_df,how="outer")
#reorder column:
clustered_df = clustered_df.reindex(columns=["Algorithm","ProofType","TotalCoinsMined","TotalCoinSupply","PC 1","PC 2","PC 3","CoinName","class"])
clustered_df.head(10)

,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,PC 1,PC 2,PC 3,CoinName,class
42,Scrypt,PoW/PoS,4.199995e+01,42,-0.337452,1.019204,-0.510428,42 Coin,0
404,Scrypt,PoW/PoS,1.055185e+09,532000000,-0.320782,1.019667,-0.510735,404Coin,0
1337,X13,PoW/PoS,2.927942e+10,314159265359,2.313123,1.816473,-0.609166,EliteCoin,0
BTC,SHA-256,PoW,1.792718e+07,21000000,-0.146745,-1.313571,0.119048,Bitcoin,1
ETH,Ethash,PoW,1.076842e+08,0,-0.153041,-1.979435,0.325009,Ethereum,1
LTC,Scrypt,PoW,6.303924e+07,84000000,-0.168538,-1.159298,-0.000968,Litecoin,1
DASH,X11,PoW/PoS,9.031294e+06,22000000,-0.390473,1.315974,-0.455447,Dash,0
XMR,CryptoNight-V7,PoW,1.720114e+07,0,-0.148064,-2.184012,0.359271,Monero,1
ETC,Ethash,PoW,1.133597e+08,210000000,-0.151482,-1.979503,0.324983,Ethereum Classic,1
ZEC,Equihash,PoW,7.383056e+06,21000000,-0.136710,-2.184359,0.377388,ZCash,1


## Visualizing Results:

In [171]:
#Create a 3D scatter plot :
fig = px.scatter_3d(
    clustered_df,
    x="PC 1",
    y="PC 2",
    z="PC 3",
    hover_name="CoinName",
    hover_data=["Algorithm"],
    color="class",
    symbol="class",
    width=850,
    )
fig.update_layout(legend=dict(x=0, y=1))
fig.show()
    

In [173]:
#Use hvplot.table to create a data table with all the current tradable cryptocurrencies:
clustered_df.hvplot.table(columns=["CoinName","Algorithm","ProofType","TotalCoinsMined","TotalCoinSupply","class"])

:Table   [CoinName,Algorithm,ProofType,TotalCoinsMined,TotalCoinSupply,class]